In [7]:
import pandas as pd

In [8]:
data = pd.read_csv('Carte.csv')
data.shape

(140, 8)

In [9]:
data.head()

,ID,Nome Carta,Mana,Forza,Effetto 1,Effetto 2,Tipo,Evocata
0,1,Nacho D'oro,4,4,Bonus,Evoca,Cibo,No
1,2,Nacho Minore,2,2,Nessun Effetto,Nessun Effetto,Cibo,Si
2,3,Riccardo Dio Del Caos,6,9,Rievoca,Bonus,Persona,No
3,4,Federica Dea Del Farro,5,7,Reset,Bonus,Persona,No
4,5,Anselmi Alchimista,2,3,Pesca,Nessun Effetto,Persona,No


In [18]:
import pandas as pd
import random
import json

# Carica i dati del file delle carte
data = pd.read_csv('Carte.csv')

# Assicurati che la colonna 'Mana' sia trattata come numerica
data['Mana'] = pd.to_numeric(data['Mana'], errors='coerce')

# Funzione per creare mazzi
# Un mazzo è equilibrato nel costo in mana e ha effetti coerenti tra di loro
def crea_mazzo(data, mazzo_size=12, mana_min=25, mana_max=48):
    mazzi = {}

    # Filtra il dataset per ottenere solo carte che rispettano i requisiti di mana
    data_filtrata = data[(data['Mana'] >= 1) & (data['Mana'] <= mana_max)].dropna(subset=['Mana'])

    # Identifica tutti gli effetti unici presenti in Effetto 1 ed Effetto 2 (escludendo "Nessun Effetto")
    effetti_unici = set(data_filtrata['Effetto 1']).union(set(data_filtrata['Effetto 2']))
    effetti_unici.discard("Nessun Effetto")

    for effetto in effetti_unici:
        # Filtra le carte che hanno l'effetto specificato in Effetto 1 o Effetto 2
        carte_con_effetto = data_filtrata[(data_filtrata['Effetto 1'] == effetto) | (data_filtrata['Effetto 2'] == effetto)]

        if len(carte_con_effetto) >= mazzo_size:
            # Mischia le carte
            carte_con_effetto = carte_con_effetto.sample(frac=1, random_state=random.randint(1, 100))

            # Seleziona un mazzo bilanciato in mana
            mazzo = bilancia_mana(carte_con_effetto, mazzo_size, mana_min, mana_max)
            if mazzo is not None:
                mazzi[effetto] = mazzo.to_dict(orient='records')

    return mazzi

# Funzione per bilanciare il mana in un mazzo
def bilancia_mana(carte, mazzo_size, mana_min, mana_max):
    for _ in range(100):  # Prova a trovare un mazzo bilanciato in 100 tentativi
        if len(carte) < mazzo_size:
            return None
        mazzo = carte.sample(n=mazzo_size, random_state=random.randint(1, 100))
        total_mana = mazzo['Mana'].sum()
        if mana_min <= total_mana <= mana_max:
            return mazzo
    return None

# Esempio di utilizzo
mazzi_creati = crea_mazzo(data)

# Controlla se sono stati creati mazzi
if mazzi_creati:
    # Salva i mazzi in un file JSON
    with open('mazzi.json', 'w', encoding='utf-8') as f:
        json.dump(mazzi_creati, f, ensure_ascii=False, indent=4)
    print("Mazzi salvati in 'mazzi.json'")
else:
    print("Nessun mazzo creato. Controlla i criteri di filtro o il dataset.")


Mazzi salvati in 'mazzi.json'
